In [1]:
#!/usr/bin/env python
import sys
import rospy
import numpy as np
from geometry_msgs.msg import Pose, PoseStamped
from sdf_mp_integration.msg import WholeBodyPose
from std_msgs.msg import Int8 

class ExperimentManager:
    def __init__(self):
        self.base_goal_pub = rospy.Publisher('move_base_simple/goal', PoseStamped, queue_size=1)
        self.robot_goal_pub = rospy.Publisher('full_goal', WholeBodyPose, queue_size=1)

        # [x, y, z], [ox, oy, oz, ow], [joints]
        self.goals = {
            0: [[0, 0, 0], [0.0, 0.0, 0.0, 1.0], [0, 0, -1.57, -1.57, 0]], 
        }
        
        self.base_goals = {
            0: [[3.5, 0, 0], [0.0, 0.0, 0.0, 1.0]], # Original diag path

        }   
        
    def go_to_start(self):
        self.robot_goal_pub.publish(self.create_robot_goal(0))

        
    def create_base_goal(self, goal_id):
        goal = self.base_goals[goal_id]
        
        goal_msg = PoseStamped()
        
        goal_msg.header.stamp = rospy.Time.now()
        goal_msg.header.frame_id = "odom"

        goal_msg.pose.position.x = goal[0][0]
        goal_msg.pose.position.y = goal[0][1]
        goal_msg.pose.position.z = goal[0][2]
        goal_msg.pose.orientation.x = goal[1][0]
        goal_msg.pose.orientation.y = goal[1][1]
        goal_msg.pose.orientation.z = goal[1][2]
        goal_msg.pose.orientation.w = goal[1][3]
        
        return goal_msg
    
    def create_robot_goal(self, goal_id):
        goal = self.goals[goal_id]
        
        goal_msg = WholeBodyPose()
        
        goal_msg.base.header.stamp = rospy.Time.now()
        goal_msg.base.header.frame_id = "odom"

        goal_msg.base.pose.position.x = goal[0][0]
        goal_msg.base.pose.position.y = goal[0][1]
        goal_msg.base.pose.position.z = goal[0][2]
        goal_msg.base.pose.orientation.x = goal[1][0]
        goal_msg.base.pose.orientation.y = goal[1][1]
        goal_msg.base.pose.orientation.z = goal[1][2]
        goal_msg.base.pose.orientation.w = goal[1][3]
        goal_msg.arm = goal[2] 
        
        return goal_msg
        
    def start_full_experiment(self, goal_id, obstacle_id):
        goal_msg = self.create_robot_goal(goal_id)
        self.robot_goal_pub.publish(goal_msg)        
        
    def start_base_experiment(self, goal_id):
        goal_msg = self.create_base_goal(goal_id)
        self.base_goal_pub.publish(goal_msg)     

In [2]:
rospy.init_node('experiment_manager_node')
experiment_manager = ExperimentManager()

In [4]:
rospy.sleep(10)
experiment_manager.start_base_experiment(0) 
